In [1]:
import pandas as pd
import praw
from praw.models import Comment
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import requests
from tqdm import tqdm
import os
from dotenv import load_dotenv, find_dotenv

# Load environment variables defined in ENV file
load_dotenv(find_dotenv())

True

## Get posts and comments from cryptocurrency subreddit

In [2]:
reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
    user_agent=os.getenv("REDDIT_USER_AGENT"),
    username=os.getenv("REDDIT_USERNAME"),
    password=os.getenv("REDDIT_PW"),
)


In [5]:
list(reddit.subreddits.default())

[Subreddit(display_name='gadgets'),
 Subreddit(display_name='sports'),
 Subreddit(display_name='gaming'),
 Subreddit(display_name='pics'),
 Subreddit(display_name='worldnews'),
 Subreddit(display_name='videos'),
 Subreddit(display_name='AskReddit'),
 Subreddit(display_name='aww'),
 Subreddit(display_name='Music'),
 Subreddit(display_name='funny'),
 Subreddit(display_name='news'),
 Subreddit(display_name='movies'),
 Subreddit(display_name='blog'),
 Subreddit(display_name='books'),
 Subreddit(display_name='europe'),
 Subreddit(display_name='history'),
 Subreddit(display_name='food'),
 Subreddit(display_name='philosophy'),
 Subreddit(display_name='Jokes'),
 Subreddit(display_name='Art'),
 Subreddit(display_name='DIY'),
 Subreddit(display_name='space'),
 Subreddit(display_name='Documentaries'),
 Subreddit(display_name='askscience'),
 Subreddit(display_name='nottheonion'),
 Subreddit(display_name='todayilearned'),
 Subreddit(display_name='gifs'),
 Subreddit(display_name='listentothis'),
 Su

In [6]:
subreddit = reddit.subreddit("cryptocurrency")

top_subs = list(subreddit.top("day"))

## Get tradable cryptocurrencies

Get the list of currently tradable cryptocurrencies from coin api and filter them to only show the ones that are
currently traded and have a dollar price

In [6]:
url = "https://rest.coinapi.io/v1/assets"
r = requests.get(url=url, params={"apiKey": "C8101760-4BDC-4EE7-9876-EA8E0EF53F4A"})

crypto_df = pd.DataFrame(r.json())

In [7]:
crypto_df = crypto_df[crypto_df["type_is_crypto"] == 1]
crypto_df = crypto_df[~crypto_df["data_trade_start"].isnull()]
crypto_df = crypto_df[~crypto_df["price_usd"].isnull()]


crypto_df

,asset_id,name,type_is_crypto,data_start,data_end,data_quote_start,data_quote_end,data_orderbook_start,data_orderbook_end,data_trade_start,data_trade_end,data_symbols_count,volume_1hrs_usd,volume_1day_usd,volume_1mth_usd,price_usd,id_icon
0,BTC,Bitcoin,1,2010-07-17,2021-05-15,2014-02-24T17:43:05.0000000Z,2021-05-15T07:18:45.4908043Z,2014-02-24T17:43:05.0000000Z,2020-08-05T14:38:38.3413202Z,2010-07-17T23:09:17.0000000Z,2021-05-15T07:15:03.6766667Z,53959,7.351389e+14,2.626787e+16,1.904027e+18,48882.234198,4caf2b16-a017-4e26-a348-2cea69c34cba
17,NIS,NIS,1,2013-03-15,2021-05-15,2018-01-24T00:42:50.1443495Z,2021-05-15T07:15:01.0200000Z,2018-01-24T00:42:50.1443495Z,2020-08-05T14:37:37.8190850Z,2013-03-15T12:48:43.0000000Z,2020-06-25T13:09:04.0000000Z,8,0.000000e+00,0.000000e+00,0.000000e+00,0.316481,NaN
18,LTC,Litecoin,1,2013-05-19,2021-05-15,2014-04-20T15:06:34.0000000Z,2021-05-15T07:18:43.8752386Z,2014-04-20T15:06:34.0000000Z,2020-08-05T14:37:59.4868435Z,2013-05-19T15:23:45.0000000Z,2021-05-15T07:15:03.6766667Z,4181,1.330712e+10,9.063569e+11,4.268167e+13,315.647300,a201762f-1499-41ef-9b84-e0742cd00e48
20,VEN,VeChain (pre-swap),1,2013-10-22,2021-05-15,2014-07-31T13:05:46.0000000Z,2021-05-15T07:18:12.8039768Z,2014-07-31T13:05:46.0000000Z,2020-08-05T14:37:58.3010000Z,2013-10-22T23:09:46.8630000Z,2021-05-15T07:15:03.6766667Z,132,2.461866e+07,3.712253e+08,2.576368e+10,0.178859,99d66a67-9bdc-4b85-84ac-2a23f912b398
21,XRP,Ripple,1,2013-11-25,2021-05-15,2014-07-31T13:05:46.0000000Z,2021-05-15T07:18:45.1984087Z,2014-07-31T13:05:46.0000000Z,2020-08-05T14:37:58.7010000Z,2013-11-25T11:54:57.9270000Z,2021-05-15T07:15:03.6766667Z,1929,6.983961e+08,2.249813e+10,9.664461e+11,1.360610,ba90bcb0-cafb-4801-ac5d-d310f47d6411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10999,PETAL,PETAL,1,2021-04-16,2021-05-15,2021-04-16T14:55:34.2114371Z,2021-05-15T07:15:01.0200000Z,NaN,NaN,2021-04-16T15:08:17.0000000Z,2021-05-15T07:15:03.6766667Z,6,1.000000e-02,1.238000e+01,3.023020e+03,0.000791,NaN
11000,RAZE,RAZE,1,2021-04-16,2021-05-15,2021-04-16T15:00:00.2426627Z,2021-05-15T07:18:13.3521331Z,NaN,NaN,2021-04-16T14:59:59.9150000Z,2021-05-15T07:15:03.6766667Z,3,2.443166e+04,6.451426e+05,2.046858e+07,0.981435,NaN
11017,NFD,NFD,1,2021-04-19,2021-05-15,2021-04-19T13:01:32.8252012Z,2021-05-15T07:15:01.0200000Z,NaN,NaN,2021-04-19T12:54:25.9610000Z,2021-05-15T07:15:03.6766667Z,1,0.000000e+00,1.118920e+03,1.714831e+04,0.001680,NaN
11093,NDAU,NDAU,1,2021-04-30,2021-05-15,2021-04-30T00:00:24.2412621Z,2021-05-15T07:15:01.0200000Z,NaN,NaN,2021-04-30T04:07:51.7000000Z,2021-05-15T07:15:03.6766667Z,2,0.000000e+00,2.415800e+02,6.857349e+04,25.101919,NaN


## Count the number of mentions of the top 100 cryptocurrencies


In [24]:
# Sort by descending trade volume
top_100_crypto = crypto_df.sort_values("price_usd", ascending=False)
crypto_ids_and_names = list(zip(list(top_100_crypto["asset_id"].astype(str)), list(top_100_crypto["name"].astype(str))))

crypto_ids_and_names

[('CASH', 'Cashcoin'),
 ('BITC', 'BITC'),
 ('POT', 'PotCoin'),
 ('VEIL', 'VEIL'),
 ('GHOST', 'GHOST'),
 ('YOBTC', 'YOBTC'),
 ('ONION', 'DeepOnion'),
 ('XUSD', 'CoinoUSD'),
 ('ZANO', 'nan'),
 ('ERG', 'nan'),
 ('INC', 'Influence Chain'),
 ('UPX', 'UPX'),
 ('DUCATO', 'DUCATO'),
 ('BTCBULL', 'BTCBULL'),
 ('OMB', 'OMB'),
 ('LTHN', 'Lethean'),
 ('DOG', 'DOG'),
 ('BPCN', 'BPCN'),
 ('BCHHG', 'nan'),
 ('EOSHG', 'nan'),
 ('BTCHG', 'nan'),
 ('DOGEBEAR', 'DOGEBEAR'),
 ('EXCHBULL', 'EXCHBULL'),
 ('DOGEHALF', 'DOGEHALF'),
 ('XRPHG', 'nan'),
 ('MATICHALF', 'MATICHALF'),
 ('YFI', 'YFI'),
 ('KSP', 'KSP'),
 ('ADAHALF', 'ADAHALF'),
 ('TBTC', 'TBTC'),
 ('ETHHG', 'nan'),
 ('WBTC', 'WBTC'),
 ('BTC', 'Bitcoin'),
 ('WBT', 'WBT'),
 ('RENBTC', 'RENBTC'),
 ('RBTC', 'RBTC'),
 ('ATOMX', 'ATOMX'),
 ('BNBHALF', 'BNBHALF'),
 ('BTCB', 'BTCB'),
 ('ADABULL', 'ADABULL'),
 ('THETAHALF', 'THETAHALF'),
 ('ETCHALF', 'ETCHALF'),
 ('DOGEBULL', 'DOGEBULL'),
 ('ETHHALF', 'ETHHALF'),
 ('BULL', 'BULL'),
 ('LUNAX', 'LUNAX'),
 ('THE

In [30]:
# Create dict structure for parsing results

parse_results = {}
for curr in crypto_ids_and_names:
    curr_id = curr[0]
    parse_results[curr_id] = {
        "mention_count": 0,
        "sentiment_score": None,
        "time": None,
        "upvotes":  0,
        "upvote_percentage": None
    }

parse_results

{'CASH': {'mention_count': 0, 'sentiment_score': None},
 'BITC': {'mention_count': 0, 'sentiment_score': None},
 'POT': {'mention_count': 0, 'sentiment_score': None},
 'VEIL': {'mention_count': 0, 'sentiment_score': None},
 'GHOST': {'mention_count': 0, 'sentiment_score': None},
 'YOBTC': {'mention_count': 0, 'sentiment_score': None},
 'ONION': {'mention_count': 0, 'sentiment_score': None},
 'XUSD': {'mention_count': 0, 'sentiment_score': None},
 'ZANO': {'mention_count': 0, 'sentiment_score': None},
 'ERG': {'mention_count': 0, 'sentiment_score': None},
 'INC': {'mention_count': 0, 'sentiment_score': None},
 'UPX': {'mention_count': 0, 'sentiment_score': None},
 'DUCATO': {'mention_count': 0, 'sentiment_score': None},
 'BTCBULL': {'mention_count': 0, 'sentiment_score': None},
 'OMB': {'mention_count': 0, 'sentiment_score': None},
 'LTHN': {'mention_count': 0, 'sentiment_score': None},
 'DOG': {'mention_count': 0, 'sentiment_score': None},
 'BPCN': {'mention_count': 0, 'sentiment_score

In [42]:
crypto_posts = list(subreddit.top("day"))

vader = SentimentIntensityAnalyzer()

comment_mentions = []

for i, post in enumerate(crypto_posts):
    post.comments.replace_more(limit=10)
    # Parse raw mention counts
    for comment in post.comments.list():

        body = comment.body
        for curr in crypto_ids_and_names:
            if curr[0] in body or curr[1] in body:

                polarity_score = vader.polarity_scores(body)
                mention = {
                    "comment": comment.id,
                    "time": comment.created_utc,
                    "currency": curr[0],
                    "pos": polarity_score["pos"],
                    "neg": polarity_score["neg"],
                    "neu": polarity_score["neu"],
                    "score": comment.score
                }
                comment_mentions.append(mention)

    if i > 3:
        break

comment_mentions

[{'comment': 'gy4zl1z',
  'time': 1621022038.0,
  'currency': 'BTC',
  'pos': 0.221,
  'neg': 0.0,
  'neu': 0.779,
  'score': 3816},
 {'comment': 'gy4zl1z',
  'time': 1621022038.0,
  'currency': 'BT',
  'pos': 0.221,
  'neg': 0.0,
  'neu': 0.779,
  'score': 3816},
 {'comment': 'gy501ge',
  'time': 1621022240.0,
  'currency': 'R',
  'pos': 0.0,
  'neg': 0.0,
  'neu': 1.0,
  'score': 788},
 {'comment': 'gy4ysjw',
  'time': 1621021690.0,
  'currency': 'F',
  'pos': 0.414,
  'neg': 0.0,
  'neu': 0.586,
  'score': 397},
 {'comment': 'gy4ysjw',
  'time': 1621021690.0,
  'currency': 'NFT',
  'pos': 0.414,
  'neg': 0.0,
  'neu': 0.586,
  'score': 397},
 {'comment': 'gy4ysjw',
  'time': 1621021690.0,
  'currency': 'LOL',
  'pos': 0.414,
  'neg': 0.0,
  'neu': 0.586,
  'score': 397},
 {'comment': 'gy524c7',
  'time': 1621023176.0,
  'currency': 'D',
  'pos': 0.162,
  'neg': 0.0,
  'neu': 0.838,
  'score': 67},
 {'comment': 'gy4z6ac',
  'time': 1621021857.0,
  'currency': 'R',
  'pos': 0.0,
  'ne

In [43]:
comment_df = pd.DataFrame(comment_mentions)
comment_df["time"] = pd.to_datetime(comment_df["time"], unit="s")
comment_df

,comment,time,currency,pos,neg,neu,score
0,gy4zl1z,2021-05-14 19:53:58,BTC,0.221,0.000,0.779,3816
1,gy4zl1z,2021-05-14 19:53:58,BT,0.221,0.000,0.779,3816
2,gy501ge,2021-05-14 19:57:20,R,0.000,0.000,1.000,788
3,gy4ysjw,2021-05-14 19:48:10,F,0.414,0.000,0.586,397
4,gy4ysjw,2021-05-14 19:48:10,NFT,0.414,0.000,0.586,397
...,...,...,...,...,...,...,...
8487,gy66oxc,2021-05-15 01:59:58,ERS,0.031,0.027,0.942,2
8488,gy66oxc,2021-05-15 01:59:58,R,0.031,0.027,0.942,2
8489,gy66oxc,2021-05-15 01:59:58,CP,0.031,0.027,0.942,2
8490,gy5cug1,2021-05-14 21:35:23,BTC,0.122,0.087,0.791,5


In [44]:
comment_df["currency"].value_counts()

D       514
BTC     206
R       168
F       168
BT      152
       ... 
PLA       1
DON       1
SYNC      1
LC        1
LX        1
Name: currency, Length: 351, dtype: int64

In [37]:

vader.polarity_scores("not so great")




{'neg': 0.637, 'neu': 0.363, 'pos': 0.0, 'compound': -0.544}